# Brick optimisation - simple fermionic cases

Created 10/102/2025

Objectives:
* Sweep over all simple fermionic cases, checking to see if we can extract the SPT and the extraction makes sense.

# Package imports

In [2]:
import sys
sys.path.append('../')

In [3]:
import h5py
from tenpy.tools import hdf5_io
import tenpy
import tenpy.linalg.np_conserved as npc

import os

In [4]:
from functools import reduce
from operator import mul

In [91]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [6]:
from SPTOptimization.SymmetryActionWithBoundaryUnitaries import SymmetryActionWithBoundaryUnitaries

from SPTOptimization.Optimizers.OneSiteSolver import OneSiteSolver

from SPTOptimization.utils import (
    to_npc_array,
    get_right_identity_environment,
    get_right_identity_environment_from_tp_tensor,
    multiply_transfer_matrices
)

from SPTOptimization.Optimizers.utils import (
    one_site_optimization_sweep_right
)

In [7]:
import re

# Load data

In [8]:
DATA_DIR = r"../data/simple_fermionic_cluster_200_site_dmrg/"

In [9]:
def parse_file_name(file_name):
    non_trivial_proj_rep = bool(int(file_name[3]))

    n1_01 = int(file_name[8])
    n1_10 = int(file_name[9])

    return (non_trivial_proj_rep, (n1_01, n1_10))

In [10]:
loaded_data = dict()

for local_file_name in os.listdir(DATA_DIR):
    f_name = r"{}/{}".format(DATA_DIR, local_file_name, ignore_unknown=False)
    with h5py.File(f_name, 'r') as f:
        data = hdf5_io.load_from_hdf5(f)

        data_info = parse_file_name(local_file_name)
        loaded_data[data_info]=data['wavefunction']

In [11]:
loaded_data

{(False, (1, 1)): <tenpy.networks.mps.MPS at 0x755897572710>,
 (False, (0, 0)): <tenpy.networks.mps.MPS at 0x75584d41e590>,
 (True, (1, 0)): <tenpy.networks.mps.MPS at 0x75584d4a6a10>,
 (True, (0, 1)): <tenpy.networks.mps.MPS at 0x75584d6f4410>,
 (True, (0, 0)): <tenpy.networks.mps.MPS at 0x75584d395b10>,
 (True, (1, 1)): <tenpy.networks.mps.MPS at 0x75584d1fae10>,
 (False, (1, 0)): <tenpy.networks.mps.MPS at 0x75584d237090>,
 (False, (0, 1)): <tenpy.networks.mps.MPS at 0x75584d2e0d50>}

# Definitons

In [12]:
MAX_VIRTUAL_BOND_DIM = 8
MAX_INTERMEDIATE_VIRTUAL_BOND_DIM = 2*MAX_VIRTUAL_BOND_DIM
# MPO bond dim?
MAX_MPO_BOND_DIM = 50

SVD_CUTOFF = 1e-3

Define bosonic symmetries. Label by the group element added.

In [13]:
np_00 = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])

np_01 = np.array([
    [0, 1, 0, 0],
    [1, 0, 0, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0]
])

np_10 = np.array([
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [1, 0, 0, 0],
    [0, 1, 0, 0]
])

np_11 = np.array([
    [0, 0, 0, 1],
    [0, 0, 1, 0],
    [0, 1, 0, 0],
    [1, 0, 0, 0]
])

In [14]:
bosonic_np_symmetries = [
    np_00,
    np_01,
    np_10,
    np_11
]

In [15]:
npc_00 = to_npc_array(np_00)
npc_01 = to_npc_array(np_01)
npc_10 = to_npc_array(np_10)
npc_11 = to_npc_array(np_11)

Define "fermionic symmetries". Just identity and JW string.

In [16]:
np_I = np.array([
    [1, 0],
    [0, 1]
])

np_JW = np.array([
    [1, 0],
    [0, -1]
])

In [17]:
fermionic_np_symmetries = [np_I, np_JW]

In [18]:
fermionic_npc_symmetries = [
    to_npc_array(X) for X in fermionic_np_symmetries
]

In [19]:
symmetry_actions = [
    [[b, f] for b in bosonic_np_symmetries]
    for f in fermionic_np_symmetries
]

In [20]:
symmetry_actions

[[[array([[1, 0, 0, 0],
          [0, 1, 0, 0],
          [0, 0, 1, 0],
          [0, 0, 0, 1]]),
   array([[1, 0],
          [0, 1]])],
  [array([[0, 1, 0, 0],
          [1, 0, 0, 0],
          [0, 0, 0, 1],
          [0, 0, 1, 0]]),
   array([[1, 0],
          [0, 1]])],
  [array([[0, 0, 1, 0],
          [0, 0, 0, 1],
          [1, 0, 0, 0],
          [0, 1, 0, 0]]),
   array([[1, 0],
          [0, 1]])],
  [array([[0, 0, 0, 1],
          [0, 0, 1, 0],
          [0, 1, 0, 0],
          [1, 0, 0, 0]]),
   array([[1, 0],
          [0, 1]])]],
 [[array([[1, 0, 0, 0],
          [0, 1, 0, 0],
          [0, 0, 1, 0],
          [0, 0, 0, 1]]),
   array([[ 1,  0],
          [ 0, -1]])],
  [array([[0, 1, 0, 0],
          [1, 0, 0, 0],
          [0, 0, 0, 1],
          [0, 0, 1, 0]]),
   array([[ 1,  0],
          [ 0, -1]])],
  [array([[0, 0, 1, 0],
          [0, 0, 0, 1],
          [1, 0, 0, 0],
          [0, 1, 0, 0]]),
   array([[ 1,  0],
          [ 0, -1]])],
  [array([[0, 0, 0, 1],
    

In [21]:
cases = dict()

for k, psi in loaded_data.items():

    for i, l in enumerate(symmetry_actions):

        for j, s in enumerate(l):
            case = SymmetryActionWithBoundaryUnitaries(
                psi,
                s*50,
                left_symmetry_index=50,
                left_boundary_unitaries=[np_I, np_00]*1,
                right_boundary_unitaries=[np_00, np_I]*1
            )

            cases[(*k, i, j)] = case

In [22]:
for c in cases.values():
    c.compute_svd_approximate_expectation()

Why do I need these again? Will need to update...

In [23]:
left_trivial_leg_charge = tenpy.linalg.charges.LegCharge(
    tenpy.linalg.charges.ChargeInfo([], []),
    [0,1],
    [[]],
    qconj=1
)

In [24]:
right_trivial_leg_charge = tenpy.linalg.charges.LegCharge(
    tenpy.linalg.charges.ChargeInfo([], []),
    [0,1],
    [[]],
    qconj=-1
)

## Functions

### Leg and label functions

In [25]:
P_LEG_LABEL_REGEX_STRING = r"^p\d*$"
p_leg_pattern = re.compile(P_LEG_LABEL_REGEX_STRING)

def is_single_physical_leg_label(label):
    return bool(p_leg_pattern.match(label))

In [26]:
def is_grouped_physical_leg_label(leg_label):
    head, *body, tail = leg_label
    if (head != '(') or (tail != ')'):
        return False

    body = ''.join(body)
    legs = body.split('.')

    return all(is_single_physical_leg_label(l) for l in legs)

In [27]:
def is_physical_leg_label(label):
    out = (
        is_single_physical_leg_label(label)
        or is_grouped_physical_leg_label(label)
    )

    return out

In [28]:
def extract_single_physical_leg_label_from_tensor(b):
    out = next(
        l for l in b.get_leg_labels()
        if is_single_physical_leg_label(l)
    )

    return out

In [29]:
def conjugate_single_physical_leg_label(label):
    return label + '*'

In [30]:
def conjugate_grouped_physical_leg_label(label):
    _, *body, _ = label
    single_labels = ''.join(body).split('.')

    conjugate_labels = [
        conjugate_single_physical_leg_label(l)
        for l in single_labels
    ]

    conjugate_body = '.'.join(conjugate_labels)
    out = '(' + conjugate_body + ')'
    
    return out

In [31]:
def conjugate_leg_label(label):
    if is_single_physical_leg_label(label):
        return conjugate_single_physical_leg_label(label)
    elif is_grouped_physical_leg_label(label):
        return conjugate_grouped_physical_leg_label(label)
    else:
        raise ValueError

In [32]:
def get_physical_leg_labels(t):
    out = [
        ll for ll in t.get_leg_labels()
        if is_physical_leg_label(ll)
    ]

    return out

### Other

In [33]:
def contract_virtual_legs(tl, tr):
    return npc.tensordot(tl, tr, ['vR', 'vL'])

In [34]:
def combine_tensors(tensors):
    out = reduce(contract_virtual_legs, tensors)

    leg_labels = [
        extract_single_physical_leg_label_from_tensor(t)
        for t in tensors
    ]

    out = out.combine_legs(leg_labels)

    return out

In [35]:
def combine_b_tensors(b_tensors):
    renamed_tensors = [
        b.replace_label('p', f'p{i}')
        for i, b in enumerate(b_tensors)
    ]

    return combine_tensors(renamed_tensors)

In [36]:
def get_identity_operator(mps_tensor):
    p_leg_label = get_physical_leg_labels(mps_tensor)[0]
    p_leg = mps_tensor.get_leg(p_leg_label)
    p_leg_label_conj = conjugate_leg_label(p_leg_label)

    out = npc.diag(
        1,
        leg=p_leg,
        dtype='complex',
        labels=[p_leg_label, p_leg_label_conj]
    )

    return out

In [37]:
def svd_reduce_split_tensor(t, max_inner_dim=MAX_VIRTUAL_BOND_DIM,
                           normalise=True, svd_cutoff=SVD_CUTOFF):
    U, S, VH = npc.svd(
        t,
        compute_uv=True,
        inner_labels=['vR', 'vL'],
        cutoff=svd_cutoff
    )

    # Truncate tensors:
    U = U[:, :max_inner_dim]
    S = S[:max_inner_dim]
    VH = VH[:max_inner_dim, :]

    if normalise:
        new_norm = np.sqrt(np.sum(S**2))
        S = S/new_norm

    """
    leg = VH.get_leg('vL')

    schmidt_values = npc.diag(S, leg, labels=['vL', 'vR'])
    """

    return U, S, VH

In [38]:
def split_combined_b(b, leftmost_schmidt_values,
                     max_virtual_bond_dim=MAX_INTERMEDIATE_VIRTUAL_BOND_DIM,
                     p_leg_labels=None):
    t = b.split_legs()

    num_sites = t.ndim - 2

    if p_leg_labels is None:
        p_leg_labels = [f'p{i}' for i in range(num_sites)]

    out_bs = list()
    out_schmidt_values = list()

    current_left_schmidt_values = leftmost_schmidt_values

    for i, ll in enumerate(p_leg_labels[:-1]):
        # In case the bond dimension has been truncated. May need to add in a
        # case if have less schmidt values than the bond dim...
        bond_dim = t.get_leg('vL').get_block_sizes()[0]
        t.iscale_axis(current_left_schmidt_values[:bond_dim], axis='vL')

        tail_legs = p_leg_labels[(i+1):]
        
        t = t.combine_legs([['vL', ll], ['vR', *tail_legs]])

        U, S, VH = svd_reduce_split_tensor(
            t,
            max_inner_dim=max_virtual_bond_dim,
            normalise=True
        )

        bl = (
            U
            .split_legs()
            .replace_label(ll, 'p')
        )
        bl.iscale_axis(1/current_left_schmidt_values[:bond_dim], axis='vL')
        bl.iscale_axis(S, axis='vR')
        bl.itranspose(['vL', 'p', 'vR'])
        out_bs.append(bl)

        out_schmidt_values.append(S)
        current_left_schmidt_values=S

        t = VH.split_legs()

    bl = t.replace_label(p_leg_labels[-1], 'p')
    bl.itranspose(['vL', 'p', 'vR'])
    out_bs.append(bl)

    return out_bs, out_schmidt_values

In [39]:
def split_b(b, max_virtual_bond_dim=MAX_INTERMEDIATE_VIRTUAL_BOND_DIM,
                     p_leg_labels=None):
    leg_label = get_physical_leg_labels(b)[0]

    if is_single_physical_leg_label(leg_label):
        return b
    elif is_grouped_physical_leg_label(leg_label):
        return split_combined_b(b, max_virtual_bond_dim, p_leg_labels)
    else:
        raise ValueError

In [40]:
def split_combined_u(u, max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM,
                     p_leg_labels=None):
    t = u.split_legs()
    num_sites = t.ndim //2
    
    if p_leg_labels is None:
        p_leg_labels = [f'p{i}' for i in range(num_sites)]

    p_leg_labels_hc = [l+'*' for l in p_leg_labels]
    out_ws = list()

    for i, (ll, ll_hc) in enumerate(zip(p_leg_labels[:-1], p_leg_labels_hc[:-1])):
        tail_legs = p_leg_labels[(i+1):]
        tail_legs_hc = p_leg_labels_hc[(i+1):]

        left_leg_labels = [ll, ll_hc]
        if 'vL' in t.get_leg_labels():
            left_leg_labels.append('vL')
    
        t = t.combine_legs([left_leg_labels, [*tail_legs, *tail_legs_hc]])

        U, S, VH = svd_reduce_split_tensor(
            t,
            max_inner_dim=max_virtual_bond_dim,
            normalise=False
        )

        U.iscale_axis(S, axis='vR')

        wl = U.split_legs().replace_labels([ll, ll_hc], ['p', 'p*'])
        #wl.itranspose(['p', 'p*'])
        out_ws.append(wl)

        t = VH.split_legs()

    wl = t.replace_labels(
        [p_leg_labels[-1], p_leg_labels_hc[-1]],
        ['p', 'p*']
    )
    #wl.itranspose(['p', 'p*'])
    out_ws.append(wl)

    out_ws[0] = (
        out_ws[0]
        .add_leg(left_trivial_leg_charge, 0, label='vL')
    )
    
    out_ws[-1] = (
        out_ws[-1]
        .add_leg(right_trivial_leg_charge, 0, label='vR')
    )

    return out_ws

In [41]:
def split_u(u, max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM,
            p_leg_labels=None):
    leg_label = get_physical_leg_labels(u)[0]

    if is_single_physical_leg_label(leg_label):
        return [unitary_to_mpo_tensor(u),]
    elif is_grouped_physical_leg_label(leg_label):
        return split_combined_u(u, max_virtual_bond_dim, p_leg_labels)
    else:
        raise ValueError

In [42]:
def unitary_to_mpo_tensor(u):
    out = (
        u
        .add_leg(left_trivial_leg_charge, 0, label='vL')
        .add_leg(right_trivial_leg_charge, 0, label='vR')
    )

    return out

In [43]:
def multiply_individual_mpo_tensors(w1, w2):
    w1 = w1.replace_labels(['vR', 'vL'], ['vR1', 'vL1'])
    w2 = w2.replace_labels(['vR', 'vL'], ['vR2', 'vL2'])

    w = (
        npc.tensordot(w1, w2, [['p'], ['p*']])
        .combine_legs([['vR1', 'vR2'], ['vL1', 'vL2']])
        .replace_labels(['(vR1.vR2)', '(vL1.vL2)'], ['vR', 'vL'])
    )

    return w

In [44]:
def group_elements(l, group_size, offset=0):
    first, rest = l[:offset], l[offset:]

    num_rest_groups = ((len(rest)-1)//group_size) + 1

    groups = [first,] if first else list()

    for i in range(num_rest_groups):
        first_index = i*group_size
        last_index = (i+1)*group_size
        groups.append(rest[first_index:last_index])

    return groups

In [45]:
def combine_grouped_b_tensors(grouped_bs):
    out = list()

    for group in grouped_bs:
        if len(group) == 1:
            out.append(group[0])
        else:
            out.append(combine_b_tensors(group))

    return out

In [46]:
def reduce_virtual_mpo_dim(wl, wr, max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM):
    # wl = wl.replace_labels(['p', 'p*'], ['pl', 'pl*'])

    wl = wl.combine_legs(['p', 'vL', 'p*'])
    wr = wr.combine_legs(['p', 'vR', 'p*'])
    w = npc.tensordot(wl, wr, ['vR', 'vL'])

    wl, S, wr = svd_reduce_split_tensor(
        w,
        max_inner_dim=max_virtual_bond_dim
    )

    wl.iscale_axis(S, axis='vR')

    wl = wl.split_legs()
    wr = wr.split_legs()

    return wl, wr

In [47]:
def multiply_mpos(top_ws, bottom_ws, max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM):
    ws = list()
    
    for w1, w2 in zip(top_ws, bottom_ws):
        w = multiply_individual_mpo_tensors(w1, w2)
        ws.append(w)


    for i in range(len(ws)-1):
        ws[i], ws[i+1] = reduce_virtual_mpo_dim(
            ws[i],
            ws[i+1],
            max_virtual_bond_dim=max_virtual_bond_dim
        )

    return ws

### Layer function

In [48]:
def optimise_layer(
        left_environment,
        top_b_tensors,
        left_schmidt_values,
        block_width,
        block_offset,
        expectations,
        unitaries=list(),
        bottom_b_tensors=None,
        num_iterations=1,
        max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM
    ):

    if bottom_b_tensors is None:
        bottom_b_tensors = top_b_tensors

    group = lambda x: group_elements(x, block_width, block_offset)
    top_grouped_bs = group(top_b_tensors)
    bottom_grouped_bs = group(bottom_b_tensors)
    grouped_schmidt_values = group(left_schmidt_values)

    top_combined_bs = combine_grouped_b_tensors(top_grouped_bs)
    bottom_combined_bs = combine_grouped_b_tensors(bottom_grouped_bs)

    new_unitaries = [
        get_identity_operator(t) for t in top_combined_bs
    ]

    expectations.append(list())

    for _ in range(num_iterations):
        exps, *_ = one_site_optimization_sweep_right(
            left_environment,
            top_combined_bs,
            new_unitaries,
            bottom_combined_bs
        )

        expectations[-1].append(exps)

    unitaries.append(new_unitaries)

    for i, u in enumerate(new_unitaries):
        b = top_combined_bs[i]
        ll = get_physical_leg_labels(b)[0]
        llh = conjugate_leg_label(ll)
    
        new_b = npc.tensordot(b, u, [[ll,], [llh,]])
    
        top_combined_bs[i] = new_b

    new_top_bs = list()
    #new_left_schmidt_values = left_schmidt_values.copy()
    new_left_schmidt_values = list()

    for b, s in zip (top_combined_bs, grouped_schmidt_values):
        """
        print(b)
        print(s)
        print('\n')
        """
        leg_label = get_physical_leg_labels(b)[0]
        if is_single_physical_leg_label(leg_label):
            new_top_bs.append(b)
            new_left_schmidt_values.extend(s)
        elif is_grouped_physical_leg_label(leg_label):
            bs, schmidt_vals = split_combined_b(
                b,
                s[0],
                max_virtual_bond_dim
            )
            new_top_bs.extend(bs)
            new_left_schmidt_values.extend(s)

    """
    for i, b in enumerate(new_top_bs):
        top_b_tensors[i] = b
    """

    return new_top_bs, new_left_schmidt_values

In [49]:
def inner_product_b_tensors(b_tensors, b_bra_tensors=None, left_environment=None,
                            right_environment=None):
    if b_bra_tensors is None:
        b_bra_tensors = b_tensors

    b = b_tensors[0]
    b_bra = b_bra_tensors[0]

    if left_environment is None:
        t = npc.tensordot(b, b_bra.conj(), [['vL',], ['vL*',]])
    else:
        t = npc.tensordot(left_environment, b, [['vR',], ['vL',]])
        t = npc.tensordot(t, b_bra.conj(), [['vR*', 'p'], ['vL*', 'p*']])

    for b, b_bra in zip(b_tensors[1:], b_bra_tensors[1:]):
        t = npc.tensordot(t, b, [['vR',], ['vL',]])
        t = npc.tensordot(t, b_bra.conj(), [['vR*', 'p'], ['vL*', 'p*']])

    if right_environment is None:
        out = npc.trace(t)
    else:
        out = npc.tensordot(t, right_environment, [['vR', 'vR*'], ['vL', 'vL*']])

    return out

In [50]:
def get_left_side_right_symmetry_environment(
    right_top_b_tensors, right_bottom_b_tensors, symmetry_transfer_matrix
    ):

    if right_bottom_b_tensors is None:
        right_bottom_b_tensors = right_top_b_tensors

    t = get_right_identity_environment_from_tp_tensor(right_top_b_tensors[-1])

    for tb, bb in zip(right_top_b_tensors[::-1], right_bottom_b_tensors[::-1]):
        t = npc.tensordot(t, tb, [['vL',], ['vR']])
        t = npc.tensordot(t, bb.conj(), [['vL*', 'p'], ['vR*', 'p*']])

    t = npc.tensordot(
        t,
        symmetry_transfer_matrix,
        [['vL', 'vL*',], ['vR', 'vR*']]
    )

    return t

In [51]:
def swap_left_right_indices(npc_array):
    left_right_pairs = {
        'vL': 'vR',
        'vR': 'vL',
        'vL*': 'vR*',
        'vR*': 'vL*'
    }

    leg_labels = npc_array.get_leg_labels()

    old_labels = [l for l in leg_labels if l in left_right_pairs]
    new_labels = [left_right_pairs[l] for l in old_labels]

    out = npc_array.replace_labels(old_labels, new_labels)

    return out

In [52]:
def two_sided_optimise_layer(
        symmetry_transfer_matrix,
        top_left_b_tensors,
        top_right_b_tensors,
        right_side_left_schmidt_values,
        left_side_right_schmidt_values,
        block_width,
        block_offset,
        right_expectations,
        left_expectations,
        bottom_left_b_tensors=None,
        bottom_right_b_tensors=None,
        left_unitaries=list(),
        right_unitaries=list(),
        num_iterations=1,
        max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM
    ):

    if bottom_left_b_tensors is None:
        bottom_left_b_tensors = top_left_b_tensors

    if bottom_right_b_tensors is None:
        bottom_right_b_tensors = top_right_b_tensors

    right_side_left_symmetry_environment = (
        get_left_side_right_symmetry_environment(
            top_left_b_tensors,
            bottom_left_b_tensors,
            swap_left_right_indices(symmetry_transfer_matrix)
        )
    )

    right_side_left_symmetry_environment = swap_left_right_indices(
        right_side_left_symmetry_environment
    )
    
    new_right_pair = optimise_layer(
        right_side_left_symmetry_environment,
        top_right_b_tensors,
        right_side_left_schmidt_values,
        block_width,
        block_offset,
        right_expectations,
        right_unitaries,
        bottom_right_b_tensors,
        num_iterations,
        max_virtual_bond_dim
    )

    new_top_right_b_tensors = new_right_pair[0]
    new_right_side_left_schmidt_values = new_right_pair[1]

    left_side_right_symmetry_environment = (
        get_left_side_right_symmetry_environment(
            new_top_right_b_tensors,
            bottom_right_b_tensors,
            symmetry_transfer_matrix
        )
    )

    left_side_right_symmetry_environment = swap_left_right_indices(
        left_side_right_symmetry_environment
    )

    new_left_pair = optimise_layer(
        left_side_right_symmetry_environment,
        top_left_b_tensors,
        left_side_right_schmidt_values,
        block_width,
        block_offset,
        left_expectations,
        left_unitaries,
        bottom_left_b_tensors,
        num_iterations,
        max_virtual_bond_dim
    )

    new_top_left_b_tensors = new_left_pair[0]
    new_left_side_right_schmidt_values = new_left_pair[1]

    return (new_left_pair, new_right_pair)

In [53]:
def initialize_brick_optimisation(symmetry_case, num_sites):
    symmetry_transfer_matrix = symmetry_case.npc_symmetry_transfer_matrix

    right_site_indices = list(range(
        symmetry_case.right_symmetry_index + 1,
        symmetry_case.right_symmetry_index + 1 + num_sites
    ))

    right_mps_tensors = [
        symmetry_case.psi.get_B(i)
        for k, i in enumerate(right_site_indices)
    ]

    right_side_left_schmidt_values = [
        symmetry_case.psi.get_SL(i)
        for i in right_site_indices
    ]

    left_site_indices = list(range(
        symmetry_case.left_symmetry_index - 1,
        symmetry_case.left_symmetry_index - 1 - num_sites,
        -1
    ))

    left_mps_tensors = [
        symmetry_case.psi.get_B(i, form='A')
        for k, i in enumerate(left_site_indices)
    ]
    
    left_mps_tensors = [swap_left_right_indices(b) for b in left_mps_tensors]

    left_side_right_schmidt_values = [
        symmetry_case.psi.get_SR(i)
        for i in left_site_indices
    ]

    right_expectations=list()
    left_expectations=list()
    left_unitaries=list()
    right_unitaries=list()

    return (
        symmetry_transfer_matrix,
        left_mps_tensors,
        right_mps_tensors,
        right_side_left_schmidt_values,
        left_side_right_schmidt_values,
        right_expectations,
        left_expectations,
        left_unitaries,
        right_unitaries
    )

# Test

In [54]:
cases.keys()

dict_keys([(False, (1, 1), 0, 0), (False, (1, 1), 0, 1), (False, (1, 1), 0, 2), (False, (1, 1), 0, 3), (False, (1, 1), 1, 0), (False, (1, 1), 1, 1), (False, (1, 1), 1, 2), (False, (1, 1), 1, 3), (False, (0, 0), 0, 0), (False, (0, 0), 0, 1), (False, (0, 0), 0, 2), (False, (0, 0), 0, 3), (False, (0, 0), 1, 0), (False, (0, 0), 1, 1), (False, (0, 0), 1, 2), (False, (0, 0), 1, 3), (True, (1, 0), 0, 0), (True, (1, 0), 0, 1), (True, (1, 0), 0, 2), (True, (1, 0), 0, 3), (True, (1, 0), 1, 0), (True, (1, 0), 1, 1), (True, (1, 0), 1, 2), (True, (1, 0), 1, 3), (True, (0, 1), 0, 0), (True, (0, 1), 0, 1), (True, (0, 1), 0, 2), (True, (0, 1), 0, 3), (True, (0, 1), 1, 0), (True, (0, 1), 1, 1), (True, (0, 1), 1, 2), (True, (0, 1), 1, 3), (True, (0, 0), 0, 0), (True, (0, 0), 0, 1), (True, (0, 0), 0, 2), (True, (0, 0), 0, 3), (True, (0, 0), 1, 0), (True, (0, 0), 1, 1), (True, (0, 0), 1, 2), (True, (0, 0), 1, 3), (True, (1, 1), 0, 0), (True, (1, 1), 0, 1), (True, (1, 1), 0, 2), (True, (1, 1), 0, 3), (True

In [55]:
test_case = cases[(False, (0,0), 0, 0)]

In [56]:
(
    symmetry_transfer_matrix,
    left_mps_tensors,
    right_mps_tensors,
    right_side_left_schmidt_values,
    left_side_right_schmidt_values,
    right_expectations,
    left_expectations,
    left_unitaries,
    right_unitaries
) = initialize_brick_optimisation(test_case, 2)

In [57]:
block_width = 2
num_iterations = 1
num_layers = 1

top_right_mpo_tensors = [right_mps_tensors,]
top_left_mpo_tensors = [left_mps_tensors,]

right_side_left_schmidt_values_list = [right_side_left_schmidt_values,]
left_side_right_schmidt_values_list = [left_side_right_schmidt_values,]

In [58]:
symmetry_transfer_matrix

<npc.Array shape=(1, 1, 1, 1) labels=['vL*', 'vL', 'vR', 'vR*']>

In [59]:
left_mps_tensors

[<npc.Array shape=(1, 2, 1) labels=['vR', 'p', 'vL']>,
 <npc.Array shape=(1, 4, 1) labels=['vR', 'p', 'vL']>]

In [60]:
right_mps_tensors

[<npc.Array shape=(1, 4, 1) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(1, 2, 1) labels=['vL', 'p', 'vR']>]

In [61]:
right_side_left_schmidt_values

[array([1.]), array([1.])]

In [62]:
left_side_right_schmidt_values

[array([1.]), array([1.])]

In [63]:
top_left_mpo_tensors[-1]

[<npc.Array shape=(1, 2, 1) labels=['vR', 'p', 'vL']>,
 <npc.Array shape=(1, 4, 1) labels=['vR', 'p', 'vL']>]

In [64]:
left_mps_tensors

[<npc.Array shape=(1, 2, 1) labels=['vR', 'p', 'vL']>,
 <npc.Array shape=(1, 4, 1) labels=['vR', 'p', 'vL']>]

In [65]:
for i in range(num_layers):
    print(f"Layer: {i}")
    
    block_offset = (block_width//2)*i % block_width

    out_quad = two_sided_optimise_layer(
        symmetry_transfer_matrix,
        top_left_mpo_tensors[-1],
        top_right_mpo_tensors[-1],
        right_side_left_schmidt_values_list[-1],
        left_side_right_schmidt_values_list[-1],
        block_width,
        block_offset,
        right_expectations,
        left_expectations,
        left_mps_tensors,
        right_mps_tensors,
        left_unitaries,
        right_unitaries,
        num_iterations
    )

    left_pair, right_pair = out_quad

    top_right_mpo_tensors.append(right_pair[0])
    right_side_left_schmidt_values_list.append(right_pair[1])
    
    top_left_mpo_tensors.append(left_pair[0])
    left_side_right_schmidt_values_list.append(left_pair[1])

Layer: 0


In [66]:
left_expectations

[[[1.0000000000000038]]]

# Sweep

In [67]:
block_width = 2
num_iterations = 1
num_layers = 1
num_sites = 2

In [68]:
left_mps_tensors = dict()
right_mps_tensors = dict()
right_side_left_schmidt_values = dict()
left_side_right_schmidt_values = dict()
right_expectations = dict()
left_expectations = dict()
left_unitaries = dict()
right_unitaries = dict()

for k, c in cases.items():
    print(f'Case {k}')

    (
        symmetry_transfer_matrix,
        current_left_mps_tensors,
        current_right_mps_tensors,
        current_right_side_left_schmidt_values,
        current_left_side_right_schmidt_values,
        current_right_expectations,
        current_left_expectations,
        current_left_unitaries,
        current_right_unitaries
    ) = initialize_brick_optimisation(c, num_sites)

    top_right_mps_tensors = [current_right_mps_tensors,]
    top_left_mps_tensors = [current_left_mps_tensors,]
    
    right_side_left_schmidt_values_list = [current_right_side_left_schmidt_values,]
    left_side_right_schmidt_values_list = [current_left_side_right_schmidt_values,]

    for i in range(num_layers):
        block_offset = (block_width//2)*i % block_width
    
        out_quad = two_sided_optimise_layer(
            symmetry_transfer_matrix,
            top_left_mps_tensors[-1],
            top_right_mps_tensors[-1],
            right_side_left_schmidt_values_list[-1],
            left_side_right_schmidt_values_list[-1],
            block_width,
            block_offset,
            current_right_expectations,
            current_left_expectations,
            current_left_mps_tensors,
            current_right_mps_tensors,
            current_left_unitaries,
            current_right_unitaries,
            num_iterations
        )
    
        left_pair, right_pair = out_quad
    
        top_right_mps_tensors.append(right_pair[0])
        right_side_left_schmidt_values_list.append(right_pair[1])
        
        top_left_mps_tensors.append(left_pair[0])
        left_side_right_schmidt_values_list.append(left_pair[1])


    left_mps_tensors[k] = top_left_mps_tensors[-1]
    right_mps_tensors[k] = top_right_mps_tensors[-1]
    right_side_left_schmidt_values[k] = right_side_left_schmidt_values_list[-1]
    left_side_right_schmidt_values[k] = left_side_right_schmidt_values_list[-1]
    right_expectations[k] = current_right_expectations
    left_expectations[k] = current_left_expectations
    left_unitaries[k] = current_left_unitaries
    right_unitaries[k] = current_right_unitaries

Case (False, (1, 1), 0, 0)
Case (False, (1, 1), 0, 1)
Case (False, (1, 1), 0, 2)
Case (False, (1, 1), 0, 3)
Case (False, (1, 1), 1, 0)
Case (False, (1, 1), 1, 1)
Case (False, (1, 1), 1, 2)
Case (False, (1, 1), 1, 3)
Case (False, (0, 0), 0, 0)
Case (False, (0, 0), 0, 1)
Case (False, (0, 0), 0, 2)
Case (False, (0, 0), 0, 3)
Case (False, (0, 0), 1, 0)
Case (False, (0, 0), 1, 1)
Case (False, (0, 0), 1, 2)
Case (False, (0, 0), 1, 3)
Case (True, (1, 0), 0, 0)
Case (True, (1, 0), 0, 1)
Case (True, (1, 0), 0, 2)
Case (True, (1, 0), 0, 3)
Case (True, (1, 0), 1, 0)
Case (True, (1, 0), 1, 1)
Case (True, (1, 0), 1, 2)
Case (True, (1, 0), 1, 3)
Case (True, (0, 1), 0, 0)
Case (True, (0, 1), 0, 1)
Case (True, (0, 1), 0, 2)
Case (True, (0, 1), 0, 3)
Case (True, (0, 1), 1, 0)
Case (True, (0, 1), 1, 1)
Case (True, (0, 1), 1, 2)
Case (True, (0, 1), 1, 3)
Case (True, (0, 0), 0, 0)
Case (True, (0, 0), 0, 1)
Case (True, (0, 0), 0, 2)
Case (True, (0, 0), 0, 3)
Case (True, (0, 0), 1, 0)
Case (True, (0, 0), 1,

## JW assumption check

In [69]:
def mod_4_to_bit(x):    
    possible_mod_4_values = list(range(4))
    assert x in possible_mod_4_values

    x0 = x % 2
    x1 = ((x - x0) // 2) % 2

    return (x0, x1)

In [70]:
def bit_to_mod_4(bit_pair):
    x0, x1 = bit_pair

    possible_mod_2_values = list(range(2))
    assert x0 in possible_mod_2_values
    assert x1 in possible_mod_2_values

    return x0 + 2*x1

In [71]:
def mod_4_to_bit_addition(x, y):
    (x0, x1) = mod_4_to_bit(x)
    (y0, y1) = mod_4_to_bit(y)

    out0 = (x0 + y0) % 2
    out1 = (x1 + y1) % 2

    out = bit_to_mod_4((out0, out1))

    return out

In [72]:
def get_n1_func(n1_01, n1_10):
    l = [0, n1_01, n1_10, (n1_01 + n1_10)%2]
    
    def f(g1, g2):
        i = mod_4_to_bit_addition(g1, g2)
        return l[i]

    return f

In [73]:
def expect_jw_necessary(n1_pair, symmetry_index):
    n1_func = get_n1_func(*n1_pair)

    return bool(n1_func(symmetry_index, 0))

In [74]:
expect_jw_necessary((1,0), 2)

False

In [75]:
expected_right_expecations = dict()

for k, v in right_expectations.items():
    _, n1_pair, jw, symmetry_index = k

    b1 = expect_jw_necessary(n1_pair, symmetry_index)
    b2 = bool(jw)

    exp = 1 if (b1 == b2) else 0

    expected_right_expecations[k] = (v[0][0][0], exp, b1, b2)

In [80]:
(
    expected_right_expecations[(False, (1, 1), 1, 1)],
    expected_right_expecations[(False, (1, 1), 0, 1)]
)

((1.1198602844520207e-17, 1, True, True),
 (3.018973491504081e-14, 0, True, False))

In [81]:
expected_left_expecations = dict()

for k, v in left_expectations.items():
    _, n1_pair, jw, symmetry_index = k

    b1 = expect_jw_necessary(n1_pair, symmetry_index)
    b2 = bool(jw)

    exp = 1 if (b1 == b2) else 0

    expected_left_expecations[k] = (v[0][0][0], exp, b1, b2)

In [82]:
(
    expected_left_expecations[(False, (1, 1), 1, 1)],
    expected_left_expecations[(False, (1, 1), 0, 1)]
)

((1.000000000000003, 1, True, True), (0.9999999275581962, 0, True, False))

# Sweep over on 4 boundary sites

In [83]:
block_width = 4
num_iterations = 1
num_layers = 1
num_sites = 4

In [84]:
left_mps_tensors = dict()
right_mps_tensors = dict()
right_side_left_schmidt_values = dict()
left_side_right_schmidt_values = dict()
right_expectations = dict()
left_expectations = dict()
left_unitaries = dict()
right_unitaries = dict()

for k, c in cases.items():
    print(f'Case {k}')

    (
        symmetry_transfer_matrix,
        current_left_mps_tensors,
        current_right_mps_tensors,
        current_right_side_left_schmidt_values,
        current_left_side_right_schmidt_values,
        current_right_expectations,
        current_left_expectations,
        current_left_unitaries,
        current_right_unitaries
    ) = initialize_brick_optimisation(c, num_sites)

    top_right_mps_tensors = [current_right_mps_tensors,]
    top_left_mps_tensors = [current_left_mps_tensors,]
    
    right_side_left_schmidt_values_list = [current_right_side_left_schmidt_values,]
    left_side_right_schmidt_values_list = [current_left_side_right_schmidt_values,]

    for i in range(num_layers):
        block_offset = (block_width//2)*i % block_width
    
        out_quad = two_sided_optimise_layer(
            symmetry_transfer_matrix,
            top_left_mps_tensors[-1],
            top_right_mps_tensors[-1],
            right_side_left_schmidt_values_list[-1],
            left_side_right_schmidt_values_list[-1],
            block_width,
            block_offset,
            current_right_expectations,
            current_left_expectations,
            current_left_mps_tensors,
            current_right_mps_tensors,
            current_left_unitaries,
            current_right_unitaries,
            num_iterations
        )
    
        left_pair, right_pair = out_quad
    
        top_right_mps_tensors.append(right_pair[0])
        right_side_left_schmidt_values_list.append(right_pair[1])
        
        top_left_mps_tensors.append(left_pair[0])
        left_side_right_schmidt_values_list.append(left_pair[1])


    left_mps_tensors[k] = top_left_mps_tensors[-1]
    right_mps_tensors[k] = top_right_mps_tensors[-1]
    right_side_left_schmidt_values[k] = right_side_left_schmidt_values_list[-1]
    left_side_right_schmidt_values[k] = left_side_right_schmidt_values_list[-1]
    right_expectations[k] = current_right_expectations
    left_expectations[k] = current_left_expectations
    left_unitaries[k] = current_left_unitaries
    right_unitaries[k] = current_right_unitaries

Case (False, (1, 1), 0, 0)
Case (False, (1, 1), 0, 1)
Case (False, (1, 1), 0, 2)
Case (False, (1, 1), 0, 3)
Case (False, (1, 1), 1, 0)
Case (False, (1, 1), 1, 1)
Case (False, (1, 1), 1, 2)
Case (False, (1, 1), 1, 3)
Case (False, (0, 0), 0, 0)
Case (False, (0, 0), 0, 1)
Case (False, (0, 0), 0, 2)
Case (False, (0, 0), 0, 3)
Case (False, (0, 0), 1, 0)
Case (False, (0, 0), 1, 1)
Case (False, (0, 0), 1, 2)
Case (False, (0, 0), 1, 3)
Case (True, (1, 0), 0, 0)
Case (True, (1, 0), 0, 1)
Case (True, (1, 0), 0, 2)
Case (True, (1, 0), 0, 3)
Case (True, (1, 0), 1, 0)
Case (True, (1, 0), 1, 1)
Case (True, (1, 0), 1, 2)
Case (True, (1, 0), 1, 3)
Case (True, (0, 1), 0, 0)
Case (True, (0, 1), 0, 1)
Case (True, (0, 1), 0, 2)
Case (True, (0, 1), 0, 3)
Case (True, (0, 1), 1, 0)
Case (True, (0, 1), 1, 1)
Case (True, (0, 1), 1, 2)
Case (True, (0, 1), 1, 3)
Case (True, (0, 0), 0, 0)
Case (True, (0, 0), 0, 1)
Case (True, (0, 0), 0, 2)
Case (True, (0, 0), 0, 3)
Case (True, (0, 0), 1, 0)
Case (True, (0, 0), 1,

## JW assumption check

In [85]:
expected_right_expecations = dict()

for k, v in right_expectations.items():
    _, n1_pair, jw, symmetry_index = k

    b1 = expect_jw_necessary(n1_pair, symmetry_index)
    b2 = bool(jw)

    exp = 1 if (b1 == b2) else 0

    expected_right_expecations[k] = (v[0][0][0], exp, b1, b2)

In [86]:
(
    expected_right_expecations[(False, (1, 1), 1, 1)],
    expected_right_expecations[(False, (1, 1), 0, 1)]
)

((1.709832498225555e-17, 1, True, True),
 (3.031386161200154e-14, 0, True, False))

In [87]:
expected_left_expecations = dict()

for k, v in left_expectations.items():
    _, n1_pair, jw, symmetry_index = k

    b1 = expect_jw_necessary(n1_pair, symmetry_index)
    b2 = bool(jw)

    exp = 1 if (b1 == b2) else 0

    expected_left_expecations[k] = (v[0][0][0], exp, b1, b2)

In [93]:
list(left_expectations.keys())[0]

(False, (1, 1), 0, 0)

In [96]:
expected_expecations = list()

for k, vl in left_expectations.items():
    vr = right_expectations[k]

    proj_rep, n1_pair, jw, symmetry_index = k

    b1 = expect_jw_necessary(n1_pair, symmetry_index)
    b2 = bool(jw)

    exp = 1 if (b1 == b2) else 0

    data = (
        proj_rep,
        n1_pair[0],
        n1_pair[1],
        jw,
        symmetry_index,
        vl[0][0][0],
        vr[0][0][0],
        exp,
        b1,
        b2
    )

    expected_expecations.append(data)

expectations_df = pd.DataFrame(
    expected_expecations,
    columns=[
        'non_trivial_proj_rep',
        'n1_01',
        'n1_10',
        'jw_string_present',
        'symmetry_action_index',
        'left_expectation',
        'right_expectation',
        'predicted_expectation',
        'predicted_jw_necessary',
        'jw_string_present_bool'
    ]
)

In [97]:
expectations_df

,non_trivial_proj_rep,n1_01,n1_10,jw_string_present,symmetry_action_index,left_expectation,right_expectation,predicted_expectation,predicted_jw_necessary,jw_string_present_bool
0,False,1,1,0,0,1.0,1.000000e+00,1,False,False
1,False,1,1,0,1,1.0,3.031386e-14,0,True,False
2,False,1,1,0,2,1.0,2.984218e-14,0,True,False
3,False,1,1,0,3,1.0,1.000000e+00,1,False,False
4,False,1,1,1,0,1.0,8.238599e-14,0,False,True
...,...,...,...,...,...,...,...,...,...,...
59,False,0,1,0,3,1.0,3.684492e-14,0,True,False
60,False,0,1,1,0,1.0,4.162429e-14,0,False,True
61,False,0,1,1,1,1.0,4.161871e-14,0,False,True
62,False,0,1,1,2,1.0,4.252660e-17,1,True,True


In [98]:
expectations_df['left_expectation']

0     1.0
1     1.0
2     1.0
3     1.0
4     1.0
     ... 
59    1.0
60    1.0
61    1.0
62    1.0
63    1.0
Name: left_expectation, Length: 64, dtype: float64

In [99]:
expectations_df['left_expectation'].describe()

count    64.000000
mean      0.999997
std       0.000010
min       0.999958
25%       1.000000
50%       1.000000
75%       1.000000
max       1.000000
Name: left_expectation, dtype: float64

In [101]:
expectations_df['right_expectation_rounded'] = expectations_df['right_expectation'].round()
expectations_df['left_expectation_rounded'] = expectations_df['left_expectation'].round()

In [102]:
expectations_df['correct'] = (
    (expectations_df['right_expectation_rounded'] == expectations_df['predicted_expectation']) &
    (expectations_df['left_expectation_rounded'] == expectations_df['predicted_expectation'])
)

In [103]:
expectations_df['correct'].value_counts()

correct
False    50
True     14
Name: count, dtype: int64

In [105]:
expectations_df[expectations_df['correct']]

,non_trivial_proj_rep,n1_01,n1_10,jw_string_present,symmetry_action_index,left_expectation,right_expectation,predicted_expectation,predicted_jw_necessary,jw_string_present_bool,right_expectation_rounded,left_expectation_rounded,correct
0,False,1,1,0,0,1.0,1.0,1,False,False,1.0,1.0,True
3,False,1,1,0,3,1.0,1.0,1,False,False,1.0,1.0,True
8,False,0,0,0,0,1.0,1.0,1,False,False,1.0,1.0,True
9,False,0,0,0,1,1.0,1.0,1,False,False,1.0,1.0,True
10,False,0,0,0,2,1.0,1.0,1,False,False,1.0,1.0,True
11,False,0,0,0,3,1.0,1.0,1,False,False,1.0,1.0,True
16,True,1,0,0,0,1.0,1.0,1,False,False,1.0,1.0,True
24,True,0,1,0,0,1.0,1.0,1,False,False,1.0,1.0,True
32,True,0,0,0,0,1.0,1.0,1,False,False,1.0,1.0,True
40,True,1,1,0,0,1.0,1.0,1,False,False,1.0,1.0,True


In [107]:
expectations_df[
    (~expectations_df['correct'])
    & (expectations_df['n1_01'] == 0)
    & (expectations_df['n1_10'] == 0)
]

,non_trivial_proj_rep,n1_01,n1_10,jw_string_present,symmetry_action_index,left_expectation,right_expectation,predicted_expectation,predicted_jw_necessary,jw_string_present_bool,right_expectation_rounded,left_expectation_rounded,correct
12,False,0,0,1,0,1.000000,1.000000e+00,0,False,True,1.0,1.0,False
13,False,0,0,1,1,1.000000,1.000000e+00,0,False,True,1.0,1.0,False
14,False,0,0,1,2,1.000000,1.000000e+00,0,False,True,1.0,1.0,False
15,False,0,0,1,3,1.000000,1.000000e+00,0,False,True,1.0,1.0,False
33,True,0,0,0,1,0.999964,3.166701e-15,1,False,False,0.0,1.0,False
34,True,0,0,0,2,0.999958,3.901250e-15,1,False,False,0.0,1.0,False
35,True,0,0,0,3,0.999999,1.577191e-14,1,False,False,0.0,1.0,False
36,True,0,0,1,0,1.000000,1.000000e+00,0,False,True,1.0,1.0,False
37,True,0,0,1,1,0.999964,3.166701e-15,0,False,True,0.0,1.0,False
38,True,0,0,1,2,0.999958,3.901250e-15,0,False,True,0.0,1.0,False


In [88]:
(
    expected_left_expecations[(False, (1, 1), 1, 1)],
    expected_left_expecations[(False, (1, 1), 0, 1)]
)

((1.0000000000000038, 1, True, True), (0.9999999282052796, 0, True, False))

In [89]:
expected_left_expecations

{(False, (1, 1), 0, 0): (1.000000000000004, 1, False, False),
 (False, (1, 1), 0, 1): (0.9999999282052796, 0, True, False),
 (False, (1, 1), 0, 2): (0.9999999995360392, 0, True, False),
 (False, (1, 1), 0, 3): (1.0000000000000049, 1, False, False),
 (False, (1, 1), 1, 0): (0.9999999935589872, 0, False, True),
 (False, (1, 1), 1, 1): (1.0000000000000038, 1, True, True),
 (False, (1, 1), 1, 2): (1.0000000000000047, 1, True, True),
 (False, (1, 1), 1, 3): (0.9999999651742371, 0, False, True),
 (False, (0, 0), 0, 0): (1.000000000000009, 1, False, False),
 (False, (0, 0), 0, 1): (1.0000000000000087, 1, False, False),
 (False, (0, 0), 0, 2): (1.0000000000000056, 1, False, False),
 (False, (0, 0), 0, 3): (1.0000000000000053, 1, False, False),
 (False, (0, 0), 1, 0): (1.000000000000009, 0, False, True),
 (False, (0, 0), 1, 1): (1.0000000000000087, 0, False, True),
 (False, (0, 0), 1, 2): (1.0000000000000056, 0, False, True),
 (False, (0, 0), 1, 3): (1.0000000000000053, 0, False, True),
 (True,

## Overlaps check

In [ ]:
len(right_expectations)

In [ ]:
def interleave_left_right_expectations(left_list, right_list):
    out = list()
    for rl, ll in zip(right_list, left_list):
        out.extend([e for l in rl for e in l])
        out.extend([e for l in ll for e in l])

    return out

In [ ]:
full_expectations = list()

for ll, rl in zip(left_expectations, right_expectations):
    full_expectations.append(list())

    for ll1, rl1 in zip(ll, rl):

        data = interleave_left_right_expectations(ll1, rl1)
        full_expectations[-1].append(data)

In [ ]:
plt.plot(full_expectations[0][0])

In [ ]:
plt.plot(full_expectations[0][1])

In [ ]:
plt.plot(full_expectations[0][2])

In [ ]:
plt.plot(full_expectations[1][0])

In [ ]:
plt.plot(full_expectations[1][1])

In [ ]:
plt.plot(full_expectations[1][2])

In [ ]:
plt.plot(full_expectations[6][2])

In [ ]:
plt.plot(full_expectations[8][0])

In [ ]:
plt.plot(full_expectations[9][0])

In [ ]:
plt.plot(full_expectations[11][0])

In [ ]:
plt.plot(full_expectations[20][0])

# Calculate SPT phase

## Function definitions

In [ ]:
def get_num_legs(unitary):
    ll = unitary.get_leg_labels()[0]

    return len(ll[1:-1].split('.'))

In [ ]:
def group_by_lengths(l, lengths):
    out = list()

    current_index = 0

    for n in lengths:
        current_group = l[current_index:current_index+n]
        out.append(current_group)

        current_index += n

    return out

In [ ]:
def multiply_unitaries_against_mps(unitaries, b_tensors, left_schmidt_values,
                                  max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM):
    site_group_lens = [get_num_legs(u) for u in unitaries]

    grouped_bs = group_by_lengths(b_tensors, site_group_lens)
    grouped_schmidt_values = group_by_lengths(
        left_schmidt_values,
        site_group_lens
    )

    combined_bs = combine_grouped_b_tensors(grouped_bs)

    for i, u in enumerate(unitaries):
        b = combined_bs[i]
        ll = get_physical_leg_labels(b)[0]
        llh = conjugate_leg_label(ll)
    
        new_b = npc.tensordot(b, u, [[ll,], [llh,]])
    
        combined_bs[i] = new_b

    new_top_bs = list()
    new_left_schmidt_values = left_schmidt_values.copy()

    for b, s in zip (combined_bs, grouped_schmidt_values):
        leg_label = get_physical_leg_labels(b)[0]
        if is_single_physical_leg_label(leg_label):
            new_top_bs.append(b)
            new_left_schmidt_values.extend(s)
        elif is_grouped_physical_leg_label(leg_label):
            bs, schmidt_vals = split_combined_b(
                b,
                s[0],
                max_virtual_bond_dim
            )
            new_top_bs.extend(bs)
            new_left_schmidt_values.extend(s)

    return new_top_bs, new_left_schmidt_values

In [ ]:
def multiply_stacked_unitaries_against_mps(unitaries, b_tensors,
    left_schmidt_values, max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM):
    out_b_tensors = b_tensors.copy()
    out_left_schmidt_values = left_schmidt_values.copy()

    for l in unitaries:
        out_b_tensors, out_left_schmidt_values = multiply_unitaries_against_mps(
            l,
            out_b_tensors,
            out_left_schmidt_values,
            max_virtual_bond_dim
        )

    return out_b_tensors, out_left_schmidt_values

In [ ]:
def mps_inner_product(
        top_tensors, bottom_tensors, left_environment):

    b_top = top_tensors[0]
    b_bottom = bottom_tensors[0]

    t = npc.tensordot(left_environment, b_bottom.conj(), (['vR*',], ['vL*',]))
    t = npc.tensordot(t, b_top, (['vR', 'p*'], ['vL', 'p']))

    for b_top, b_bottom in zip(top_tensors[1:], bottom_tensors[1:]):
        t = npc.tensordot(t, b_bottom.conj(), (['vR*',], ['vL*',]))
        t = npc.tensordot(t, b_top, (['vR', 'p*'], ['vL', 'p']))

    e = npc.trace(t, 'vR', 'vR*')

    return e

In [ ]:
def calculate_proj_rep_phase(state_g, left_schmidt_values_g, unitary_h,
                             state_gh,
                             left_environment,
                             max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM):
    prod_state, _ = multiply_stacked_unitaries_against_mps(
        unitary_h,
        state_g,
        left_schmidt_values_g,
        max_virtual_bond_dim
    )

    exp = mps_inner_product(prod_state, state_gh, left_environment)

    return exp/(np.abs(exp))

## Group data definitions

In [ ]:
from itertools import combinations

In [ ]:
symmetry_labels = [
    'II',
    'IX',
    'XI',
    'XX'
]

In [ ]:
symmetry_combination_labels = list(combinations(symmetry_labels, 2))

In [ ]:
symmetry_combination_labels

In [ ]:
e, a, b, c = symmetry_labels

In [ ]:
group_products = [
    (a, b, c),
    (b, a, c),
    (a, c, b),
    (c, a, b),
    (c, b, a),
    (b, c, a)
]

## Get right phases

In [ ]:
def get_proj_rep_phases(mps_tensors, left_schmidt_values, unitaries,
                        left_environment):
    mps_dict = {
        k: v for k, v in zip(symmetry_labels[1:], mps_tensors)
    }

    schmidt_vals_dict = {
        k: v for k, v in zip(symmetry_labels[1:], left_schmidt_values)
    }

    unitaries_dict = {
        k: v for k, v in zip(symmetry_labels[1:], unitaries)
    }

    out_phases = list()

    for a, b, c in group_products:
        phase = calculate_proj_rep_phase(
            mps_dict[a],
            schmidt_vals_dict[a],
            unitaries_dict[b],
            mps_dict[c],
            left_environment,
            max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM
        )

        out_phases.append(phase)

    return out_phases

In [ ]:
right_proj_rep_phases = list()

iterable = zip(
    right_mps_tensors,
    right_side_left_schmidt_values,
    right_unitaries,
    cases
)
for mps, schmidt, unitaries, case_list in iterable:
    phases = get_proj_rep_phases(
        mps,
        schmidt,
        unitaries,
        case_list[0].right_projected_symmetry_state
    )

    right_proj_rep_phases.append(phases)

right_proj_rep_phases = np.array(right_proj_rep_phases)

In [ ]:
right_gauge_invariant_phases = (
    right_proj_rep_phases[:,::2]/
    right_proj_rep_phases[:,1::2]
)

In [ ]:
right_gauge_invariant_phases[:10]

In [ ]:
right_gauge_invariant_phases[11:]

In [ ]:
left_proj_rep_phases = list()

iterable = zip(
    left_mps_tensors,
    left_side_right_schmidt_values,
    left_unitaries,
    cases
)
for mps, schmidt, unitaries, case_list in iterable:
    phases = get_proj_rep_phases(
        mps,
        schmidt,
        unitaries,
        swap_left_right_indices(case_list[0].left_projected_symmetry_state)
    )

    left_proj_rep_phases.append(phases)

left_proj_rep_phases = np.array(left_proj_rep_phases)

In [ ]:
left_gauge_invariant_phases = (
    left_proj_rep_phases[:,::2]/
    left_proj_rep_phases[:,1::2]
)

In [ ]:
left_gauge_invariant_phases[:10]

In [ ]:
left_gauge_invariant_phases[11:]

# Conclusion
Looks good!